## Poject: Thu thập, phân tích và lựa chọn khách sạn phù hợp cho chuyển đi.

### Thao tác Selenium và BeautifulSub lên web Booking.com

In [5]:
import urllib.parse
import pandas as pd
import requests
import math
import time 
import numpy as np
from selenium_provider import Selenium
from selenium_provider import get_into
import requests
from bs4 import BeautifulSoup

- Nhập thông tin cần thiết của địa điểm.

In [6]:
# DiaDiem = 'Vung Tau'
NgayDB = '1'
NgayKT = '3'
Thang = '12'
Nam = '2023'
SoTrang = 5 # Số trang cần cào về.

In [7]:
cursor = Selenium()

- Hành động tìm kiếm theo địa điểm.

In [8]:
url = f"https://www.booking.com/"
cursor.get_url(url)
time.sleep(5)
try:
    cursor.element_click('//*[@id="b2indexPage"]/div/div/div/div/div[1]/div[1]/div/button')
except:
    pass

In [ ]:
DiaDiem

In [10]:
cursor.input('//*[@id=":re:"]', DiaDiem)
time.sleep(1)
cursor.element_click('//*[@id="autocomplete-result-0"]/div/div/span')

In [11]:
Month = ['0', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [12]:
# Pass trang chọn thời gian
MY = Month[int(Thang)] + ' ' + Nam
while (True):
    if cursor.get_element('//*[@id="calendar-searchboxdatepicker"]/div/div[1]/div/div[1]/h3').text == MY:
        break
    cursor.element_click('//*[@id="calendar-searchboxdatepicker"]/div/div[1]/button')

In [13]:
dd = []
for i in range(1,6):
    dd =dd +(cursor.get_list_element('//*[@id="calendar-searchboxdatepicker"]/div/div[1]/div/div[1]/table/tbody/tr[' + str(i) + ']/td'))
for i in dd:
    if i.text == NgayDB:
        i.click()
    if i.text == NgayKT:
        i.click()
        break

In [14]:
# search
cursor.element_click('//*[@id="indexsearch"]/div[2]/div/form/div[1]/div[4]/button')

### Cào dữ liệu.

* Note: Xem xét tổng hợp các trang khách sạn.

In [15]:
df = pd.DataFrame(columns=['name', 'star', 'number_of_reviewer', 'price', 'cost', 'discount', 'longitude', 'latitude', 'score_level', 'address', 'link'])
df = df.drop(index=df.index)

In [ ]:
last_page = cursor.get_list_element('//*[@id="bodyconstraint-inner"]/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[2]/ol/li')
last_page = int(last_page[-1].text)
try: 
    for j in range(2, last_page+2):
        # Lấy con trỏ tới các thẻ chứa các bài
        ls_room_element = cursor.get_list_element('//*[@id="bodyconstraint-inner"]/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[3]/div')
        for i in ls_room_element:
            if len(i.text) > 100:
                name = get_into(i, './/div[1]//div[2]//div//div[1]//div//div[1]//div//div[1]//div//h3//a//div[1]').text
                try: star = float(get_into(i, './/div[1]//div[2]//div//div[1]//div//div[2]//div//div//div//a//span//div//div[1]').text)
                except: star = np.nan
                try: number_of_reviewer = int(get_into(i, './/div[1]//div[2]//div//div[1]//div//div[2]//div//div//div//a//span//div//div[2]//div[2]').text.split()[0])
                except: number_of_reviewer = np.nan
                try:
                    price_tmp = get_into(i, './/div[1]//div[2]//div//div[2]//div[2]//div//div[1]//span')
                except:
                    price_tmp = get_into(i, './/div[1]//div[2]//div//div[3]//div[2]//div//div[1]//span')
                price_spl = price_tmp.text.split()
                if len(price_spl)<= 2:
                    price = int(price_spl[1].replace(",", ""))
                    cost = price
                    discount = 0
                else: 
                    cost = int(price_spl[1].replace(",", "").replace("VND", ""))
                    price = price_spl[2]
                    comma_position = price.rfind(",") + 4
                    try: discount = int(price_spl[3][0: len(price_spl[3]) - 1])
                    except: 
                        try: discount = int(price[comma_position: len(price) - 1])
                        except: discount = 0
                    price = int(price[0:comma_position].replace(",", ""))
                link = get_into(i, './/div[1]//div[1]//div//a').get_attribute("href")

                #Beautifulsub
                try: 
                    response = requests.get(link)
                    soup = BeautifulSoup(response.content, 'html.parser')
                    content = soup.find('p', class_ = 'address address_clean', id = 'showMap2').find('a')
                    coordinate = content.get('data-atlas-latlng').split(',')
                    long, lat = float(coordinate[0]), float(coordinate[1])
                    content = soup.find('p', class_ = 'address address_clean', id = 'showMap2').find('span', rel="14")
                    address = content.text
                    try: 
                        content = soup.find('div', class_ = "abf093bdfe e9951e18a7")
                        score_level = content.text.split(' ')[0]
                    except: 
                        score_level = 'medium'

                    new_row = pd.Series([name, star, number_of_reviewer, price, cost, discount, long, lat, score_level, address, link], index=df.columns)
                    df.loc[len(df)] = new_row
                except: pass

        list_page = cursor.get_list_element('//*[@id="bodyconstraint-inner"]/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[2]/ol/li')
        for k in list_page:
            try: 
                if str(k.text) == str(j): k.click()
            except: break   
            
        # Số trang cần cào.
        if j == SoTrang: break
        time.sleep(2)
except: pass

### Phân tích & xử lý dữ liệu.

In [13]:
df

,name,star,number_of_reviewer,price,cost,discount,longitude,latitude,score_level,address,link
0,The Sóng Căn Hộ Cho Thuê Full Tiện Ích,8.3,8,2592000,4320000,40,10.350025,107.095863,High,"\n28 Đường Thi Sách, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/the-song-can-...
1,Sun Hill Hotel,9.3,NaN,1047600,2327999,55,10.335941,107.081589,High,"\nH60 Phan Chu Trinh, Phường 2, Thành Phố Vũng...",https://www.booking.com/hotel/vn/sun-hill.html...
2,The Song Vung Tau Near Beach,8.5,206,1939601,3128388,38,10.350071,107.09593,medium,"\n28 Đường Thi Sách, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/the-song-vung...
3,CAROLINE SEA HOTEL,7.9,241,664077,664077,0,10.335861,107.086765,medium,"\n151 Đường Hoàng Hoa Thám, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/caroline-sea....
4,The Sóng Vũng Tàu Căn Hộ Du Lịch Ven Biển,9.3,22,2100000,4200000,50,10.350204,107.095927,High,"\n28 Đường Thi Sách, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/the-song-apar...
...,...,...,...,...,...,...,...,...,...,...,...
245,Nhà sát biển Bãi Sau 093-863-2325,8.0,3,1530000,1530000,0,10.329774,107.086611,High,"\n145 Phan Chu Trinh, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/2pn-80m2-sat-...
246,Sea Memory Hotel,9.0,490,675000,1350000,50,10.337685,107.082835,High,"\n26 phan huy ích,Khu A Chau,Phường 2, Vung Ta...",https://www.booking.com/hotel/vn/sea-memory-vu...
247,Petro House Hotel,8.1,127,1982280,1982280,0,10.349679,107.074969,medium,"\n63 Tran Hung Dao Street, Ward 1, Vung Tau, V...",https://www.booking.com/hotel/vn/petro-house.h...
248,SV Boutique Resort,8.3,209,3895000,3895000,0,10.330235,107.077382,High,"\n66/15 Hạ Long, Phường 2, Thành Phố Vũng Tàu,...",https://www.booking.com/hotel/vn/leman-cap-res...


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250 entries, 0 to 249
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   name                250 non-null    object
 1   star                224 non-null    object
 2   number_of_reviewer  215 non-null    object
 3   price               250 non-null    object
 4   cost                250 non-null    object
 5   discount            250 non-null    object
 6   longitude           250 non-null    object
 7   latitude            250 non-null    object
 8   score_level         250 non-null    object
 9   address             250 non-null    object
 10  link                250 non-null    object
dtypes: object(11)
memory usage: 23.4+ KB


In [15]:
df['cost'] = df['cost'].astype(int)
df['price'] = df['price'].astype(int)
df['discount'] = df['discount'].astype(int)

df['longitude'] = df['longitude'].astype(float)
df['latitude'] = df['latitude'].astype(float)

In [16]:
df[df['name'].duplicated() == True]

,name,star,number_of_reviewer,price,cost,discount,longitude,latitude,score_level,address,link
25,3H Grand Hotel,9.5,471,1238448,2251724,45,10.330074,107.084800,High,"\n274 Phan Chu Trinh, Phường 2, Thành Phố Vũng...",https://www.booking.com/hotel/vn/3h-grand.html...
29,The Sóng Vũng Tàu - Căn Hộ Du Lịch Ven Biển,8.8,77,2209660,4419320,50,10.350164,107.096167,High,"\n28 Đường Thi Sách, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/the-song-vung...
35,CĂN HỘ CONDOTEL 5 SAO THE SONG VŨNG TÀU - Apar...,8.3,162,1805000,3610000,50,10.350145,107.096495,High,"\n28 Đường Thi Sách, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/can-ho-the-so...
54,CONDOTEL THE SÓNG AN GIA VŨNG TÀU APARTMENT,9.5,13,1800000,3000000,40,10.350025,107.095863,High,"\n28 Đường Thi Sách 36, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/condotel-the-...
60,Milan Homestay - Milanesa Hotel and Apartment,8.2,NaN,712000,712000,0,10.340502,107.076347,medium,"\n 149 Vo Thi Sau, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/vung-tau-melo...
79,The Song Yuki Apartment,8.5,207,3780000,12600000,70,10.350071,107.095930,High,"\nThe Song building, Block A, tsc-28.18, 28 Th...",https://www.booking.com/hotel/vn/the-song-ana-...
81,Milanesa Hotel and Apartment,10.0,3,676200,1380000,51,10.340516,107.076262,High,"\n5A Đinh Tiên Hoàng Phường 2, Vung Tau, Vietn...",https://www.booking.com/hotel/vn/milanesa-and-...
85,CONDOTEL 5 SAO THE SÓNG VŨNG TÀU Mr VƯƠNG,9.1,112,1846900,2530000,27,10.350071,107.095930,High,"\n28 Đường Thi Sách, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/can-ho-5-sao-...
104,Căn hộ The Sóng - Your Homestay,8.8,33,2040000,10200000,80,10.350162,107.096406,High,"\n149 Võ Thị Sáu 25, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/the-melody-pr...
110,The Song Premium Apartment,8.6,238,2450000,9800000,75,10.350034,107.096113,medium,"\n28 Đường Thi Sách, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/can-ho-the-so...


In [17]:
att = ['name', 'address']
df = df.drop(df[df[att].duplicated() == True].index)

In [18]:
df['star'] = df['star'].fillna(df['star'].mean())
df['number_of_reviewer'] = df['number_of_reviewer'].fillna(0)

### Ghi vào file csv và trực quan bằng Power BI.

In [21]:
df.to_csv('Raw_data/{DiaDiem}.csv', index=False)

In [22]:
df

,name,star,number_of_reviewer,price,cost,discount,longitude,latitude,score_level,address,link
0,The Sóng Căn Hộ Cho Thuê Full Tiện Ích,8.300000,8,2592000,4320000,40,10.350025,107.095863,High,"\n28 Đường Thi Sách, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/the-song-can-...
1,Sun Hill Hotel,9.300000,0,1047600,2327999,55,10.335941,107.081589,High,"\nH60 Phan Chu Trinh, Phường 2, Thành Phố Vũng...",https://www.booking.com/hotel/vn/sun-hill.html...
2,The Song Vung Tau Near Beach,8.500000,206,1939601,3128388,38,10.350071,107.095930,medium,"\n28 Đường Thi Sách, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/the-song-vung...
3,CAROLINE SEA HOTEL,7.900000,241,664077,664077,0,10.335861,107.086765,medium,"\n151 Đường Hoàng Hoa Thám, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/caroline-sea....
4,The Sóng Vũng Tàu Căn Hộ Du Lịch Ven Biển,9.300000,22,2100000,4200000,50,10.350204,107.095927,High,"\n28 Đường Thi Sách, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/the-song-apar...
...,...,...,...,...,...,...,...,...,...,...,...
245,Nhà sát biển Bãi Sau 093-863-2325,8.000000,3,1530000,1530000,0,10.329774,107.086611,High,"\n145 Phan Chu Trinh, Vung Tau, Vietnam\n",https://www.booking.com/hotel/vn/2pn-80m2-sat-...
246,Sea Memory Hotel,9.000000,490,675000,1350000,50,10.337685,107.082835,High,"\n26 phan huy ích,Khu A Chau,Phường 2, Vung Ta...",https://www.booking.com/hotel/vn/sea-memory-vu...
247,Petro House Hotel,8.100000,127,1982280,1982280,0,10.349679,107.074969,medium,"\n63 Tran Hung Dao Street, Ward 1, Vung Tau, V...",https://www.booking.com/hotel/vn/petro-house.h...
248,SV Boutique Resort,8.300000,209,3895000,3895000,0,10.330235,107.077382,High,"\n66/15 Hạ Long, Phường 2, Thành Phố Vũng Tàu,...",https://www.booking.com/hotel/vn/leman-cap-res...
